In [1]:
!pip install tangelo-gc

try:
    import os
except ImportError:
    print ("os error")

!pip install PySCF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 598.9/598.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pubchempy: filename=PubChemPy-1.0.4-py3-none-any.whl size=13820 sha256=0792551cb9462a53e0c555412e6ba95f95b5c5aa1ae2bc54ff1e3b032b153d2a
  Stored in directory: /root/.cache/pip/wheels/90/7c/45/18a0671e3c3316966ef7ed9ad2b3f3300a7e41d3421a44e799
Successfully built pubchempy
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 MB 11.3 MB/s eta 0:00:00


In [2]:
###
from tangelo.toolboxes.molecular_computation.frozen_orbitals import get_orbitals_excluding_homo_lumo
from tangelo import SecondQuantizedMolecule as SQMol

f2=  """F 0.  0. 0.
         F 1.412 0. 0. """

# freeze some
fo = [0, 1, 2, 3]+[i for i in range(10,28)]

# Runs RHF calculation
mol_f2 = SQMol(f2, q=0, spin=0, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
    print(f"{i+1:3d}{mol_f2.mo_energies[i]: 9.4f}  {mol_f2.mo_symm_labels[i]}   {int(mol_f2.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_f2.n_active_electrons}")
print(f"Number of active orbtials: {mol_f2.n_active_mos}")

  #  Energy  Symm Occ
  1 -26.4268  A1g   2
  2 -26.4266  A1u   2
  3  -1.7567  A1g   2
  4  -1.4893  A1u   2
  5  -0.8019  E1uy   2
  6  -0.8019  E1ux   2
  7  -0.7419  A1g   2
  8  -0.6570  E1gx   2
  9  -0.6570  E1gy   2
 10   0.1082  A1u   0
Number of active electrons: 10
Number of active orbtials: 6


In [3]:
print("Importing VQE, Ansatze")
from tangelo.algorithms.variational import VQESolver, BuiltInAnsatze
print("Importing FCI")
from tangelo.algorithms.classical import FCISolver

algorithm_resources = dict()

# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_Li2, "ansatz": BuiltInAnsatze.UCCSD}
print("Solving")
vqe_solver = VQESolver(vqe_options)
print("Building")
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")
print(f"CASCI energy = {FCISolver(mol_f2).simulate()}")
algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

Importing VQE, Ansatze
Importing FCI
Solving
Building

 Ground Singlet state
VQE energy = -198.72875921024064
CASCI energy = -198.72875921708655


In [8]:
# Add initial VQE optimal circuit to the deflation circuits list
deflation_circuits = [vqe_solver.optimal_circuit.copy()]

dict_vqe = {}
for i in range(3):
    vqe_options = {"molecule": mol_f2, "ansatz": BuiltInAnsatze.UpCCGSD,
                   "deflation_circuits": deflation_circuits, "deflation_coeff": 18}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    vqe_energy = vqe_solver.simulate()
    print(f"Excited state #{i+1} \t VQE energy = {vqe_energy}")
    algorithm_resources[f"vqe_deflation_state_{i+1}"] = vqe_solver.get_resources()
    dict_vqe[i+1] =  vqe_energy
    deflation_circuits.append(vqe_solver.optimal_circuit.copy())

KeyboardInterrupt: 

In [27]:
# Not running, lets try it with another method
###

f2=  """F 0.  0. 0.
        F 1.412 0. 0. """

# freeze some
fo = [0, 1, 2, 3]+[i for i in range(14,28)]

# Runs RHF calculation
mol_f2_double = SQMol(f2, q=0, spin=8, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
    print(f"{i+1:3d}{mol_f2_double.mo_energies[i]: 9.4f}  {mol_f2_double.mo_symm_labels[i]}   {int(mol_f2_double.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_f2_double.n_active_electrons}")
print(f"Number of active orbtials: {mol_f2_double.n_active_mos}")

algorithm_resources = dict()


print(f"CASCI energy = {FCISolver(mol_f2_double).simulate()}")


# Ground state energy calculation with VQE, reference values with FCI
vqe_options = {"molecule": mol_f2_double, "ansatz": BuiltInAnsatze.UCCSD}
print("Solving")
vqe_solver = VQESolver(vqe_options)
print("Building")
vqe_solver.build()
vqe_energy = vqe_solver.simulate()
print("\n Ground Singlet state")
print(f"VQE energy = {vqe_energy}")


algorithm_resources["vqe_ground_state"] = vqe_solver.get_resources()

  #  Energy  Symm Occ
  1 -26.4802  A1g   2
  2 -26.4801  A1u   2
  3  -1.8529  A1g   2
  4  -1.5751  A1u   2
  5  -0.9509  A1g   2
  6  -0.5948  E1uy   1
  7  -0.5948  E1ux   1
  8  -0.4375  E1gx   1
  9  -0.4375  E1gy   1
 10  -0.3326  A1u   1
Number of active electrons: 10
Number of active orbtials: 10
CASCI energy = -194.6452941627634
Solving
Building


TypeError: 'numpy.bool_' object does not support item assignment

In [31]:
f2=  """F 0.  0. 0.
        F 1.412 0. 0. """

# freeze some
fo = [0, 1, 2, 3]+[i for i in range(14,28)]

# Runs RHF calculation
mol_f2_double = SQMol(f2, q=0, spin=4, basis='6-31g(d,p)', frozen_orbitals=fo, symmetry=True)

print("  #  Energy  Symm Occ")
for i in range(10):
    print(f"{i+1:3d}{mol_f2_double.mo_energies[i]: 9.4f}  {mol_f2_double.mo_symm_labels[i]}   {int(mol_f2_double.mo_occ[i])}")

# Active electrons, Active orbitals
print(f"Number of active electrons: {mol_f2_double.n_active_electrons}")
print(f"Number of active orbtials: {mol_f2_double.n_active_mos}")

algorithm_resources = dict()


print(f"CASCI energy = {FCISolver(mol_f2_double).simulate()}")

  #  Energy  Symm Occ
  1 -26.4949  A1g   2
  2 -26.4948  A1u   2
  3  -1.8317  A1g   2
  4  -1.5463  A1u   2
  5  -0.8904  A1g   2
  6  -0.7882  E1uy   2
  7  -0.7882  E1ux   2
  8  -0.4594  E1gx   1
  9  -0.4594  E1gy   1
 10  -0.2703  A1u   1
Number of active electrons: 10
Number of active orbtials: 10
CASCI energy = -197.3474839460521


In [42]:
# so excited states for fluorine have energy:

In [43]:
ground = 198.72875921708655
ex1 = 197.3474839460521
ex2 = 194.6452941627634

delta1 = ground-ex1
delta2 = ground-ex2
delta3 = ex1-ex2

In [44]:
print(delta1, delta2, delta3) # for fluoride

1.3812752710344398 4.083465054323142 2.702189783288702
